# Example Fonctionelle d'annimation pour la semaine 749

In [17]:
from ClementAnalyse_animation_work import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cmasher as cmr
def animate_data(
    test,
    xedge_setting,
    yedge_setting,
    step=20000,
    cmap=cmr.cosmic,
    vmax=25,
    vmin=0,
    begin=0,
    fps= 2
    ):
    interval = 1000/fps
    # Define the initial values
    begin = begin
    end = begin + step
    bins = (xedge_setting, yedge_setting)
    all_counts = []

    # Define the figure and axis objects
    fig, ax = plt.subplots()
    cbar = None

    # Define the update function that will be called for each frame of the animation
    def update(frame):
        nonlocal cbar, all_counts, begin, end
        if cbar is not None:
            cbar.remove()

        # Extract the slice of the data for this frame
        begin = frame * step
        end = begin + step
        slice = test[(test.TIME > begin) & (test.TIME < end)]

        # Compute the histogram for this slice
        count, _, _ = np.histogram2d(slice.L_cut, slice.B_cut, bins=bins)

        # Add the counts to the list of all counts
        all_counts.append(count)

        # Plot the current frame
        ax.clear()
        pcm = ax.imshow(all_counts[frame].T, extent=[xedge_setting[0], xedge_setting[-1], yedge_setting[0], yedge_setting[-1]],
                        origin='lower', cmap=cmap, vmax=vmax,vmin=vmin, interpolation='antialiased', aspect='equal')
        ax.set_title(f'Time : {np.round(slice.TIME.min()/3600)}H -> {np.round((slice.TIME.max()/3600))} H')

        # Add a colorbar to the plot
        cbar = fig.colorbar(pcm)

    # Create the animation object and start the animation
    num_frames = int(test.TIME.max()+1)//step
    ani = FuncAnimation(fig, update, frames=num_frames, interval=interval)
    plt.show()
    return ani

In [18]:
%matplotlib qt

In [19]:
w749_path = "HauteEnergie/Data/source_observation/lat_photon_weekly_w749_p305_v001.fits" #Chemin du fichier FITS
data = GammaFitsPandas(w749_path)
week = data.events
data.keep_bin_edge
xedge_setting = data.xedges_tot
yedge_setting = data.yedges_tot
plt.close('all')

100%|██████████| 301531/301531 [00:00<00:00, 4267674.99it/s]


In [23]:
animate_data(week,xedge_setting,yedge_setting,step=12*3600,vmax=50,vmin=-1,begin=0,fps=10)